# 🖼️ Trellis NIM - Image to 3D Generation

Transform your images into 3D models using NVIDIA Trellis NIM!

## Prerequisites:
- Run the `trellis_nim_simple.ipynb` notebook first to deploy the NIM container
- Make sure the Trellis NIM container is running

## Steps:
0. **Install Dependencies** - Install required Python packages
1. **Container Check** - Verify Trellis NIM is running
2. **Upload Image** - Select and upload your image
3. **Generate 3D Model** - Convert image to 3D GLB file

---


## Step 0: Install Required Dependencies

First, let's install the necessary Python packages for image upload and processing:


In [ ]:
import sys

print("📦 Installing required packages...")

# Install required packages
required_packages = [
    "ipywidgets",
    "Pillow", 
    "requests"
]

for package in required_packages:
    print(f"Installing {package}...")
    !{sys.executable} -m pip install {package} -q
    
print("✅ All packages installed successfully!")
print("💡 If running in JupyterLab, you may need to restart the kernel and enable widget extensions")
print("💡 Run: jupyter labextension install @jupyter-widgets/jupyterlab-manager")


## Step 1: Check NIM Container Connectivity

Now let's verify that the Trellis NIM container is running and accessible:


In [ ]:
import requests
import json
import base64
import subprocess
from IPython.display import display, HTML, Image as IPImage
import ipywidgets as widgets
from ipywidgets import FileUpload, Button, Output, VBox, HBox
import io
from PIL import Image
import os

print("🔍 Checking NIM container status...")

# Check if container is running
container_check = !docker ps | grep nim-server
if not container_check:
    print("❌ Trellis NIM container is not running!")
    print("💡 Please run the 'trellis_nim_simple.ipynb' notebook first to deploy the container")
    print("💡 Or start it manually with the docker run command from that notebook")
else:
    print("✅ NIM container is running")
    print(f"Container info: {container_check[0]}")

# Test connectivity
print("\n🌐 Testing NIM container connectivity...")
try:
    response = requests.get("http://localhost:8000/health", timeout=5)
    if response.status_code == 200:
        print("✅ NIM container is responding to health checks")
    else:
        print(f"⚠️ NIM container responded with status code: {response.status_code}")
except requests.RequestException as e:
    print(f"❌ Cannot connect to NIM container: {e}")
    print("💡 Make sure the container is fully initialized (may take 2-5 minutes after start)")

print("\n📊 Recent container logs:")
!docker logs nim-server --tail 5


## Step 2: Image Upload Interface

Upload an image that you want to convert to a 3D model:


In [ ]:
# Create image upload widget
upload_widget = FileUpload(
    accept='.jpg,.jpeg,.png,.bmp,.tiff,.webp',  # Accept common image formats
    multiple=False,  # Single file upload
    description='Choose Image'
)

# Output widget to display uploaded image
image_display = Output()

# Global variable to store uploaded image data
uploaded_image_data = None
uploaded_image_name = None

def on_upload_change(change):
    global uploaded_image_data, uploaded_image_name
    
    with image_display:
        image_display.clear_output()
        
        if upload_widget.value:
            # Get the uploaded file
            uploaded_file = list(upload_widget.value.values())[0]
            uploaded_image_name = list(upload_widget.value.keys())[0]
            uploaded_image_data = uploaded_file['content']
            
            print(f"📁 Uploaded: {uploaded_image_name}")
            print(f"📊 File size: {len(uploaded_image_data) / 1024:.2f} KB")
            
            # Display the uploaded image
            try:
                image = Image.open(io.BytesIO(uploaded_image_data))
                
                # Resize image for display if it's too large
                display_image = image.copy()
                if display_image.width > 400 or display_image.height > 400:
                    display_image.thumbnail((400, 400), Image.Resampling.LANCZOS)
                
                # Convert to bytes for display
                img_buffer = io.BytesIO()
                display_image.save(img_buffer, format='PNG')
                img_buffer.seek(0)
                
                display(IPImage(data=img_buffer.getvalue()))
                print(f"📏 Original dimensions: {image.width} x {image.height}")
                print(f"🎨 Image format: {image.format}")
                print("✅ Image ready for 3D generation!")
                
            except Exception as e:
                print(f"❌ Error processing image: {e}")
        else:
            uploaded_image_data = None
            uploaded_image_name = None
            print("No image selected")

# Attach the event handler
upload_widget.observe(on_upload_change, names='value')

# Display the upload interface
print("📤 Select an image to upload:")
print("Supported formats: JPG, PNG, BMP, TIFF, WebP")
print("")
display(VBox([upload_widget, image_display]))


## Step 3: Generate 3D Model from Image

Convert your uploaded image into a 3D model:


In [ ]:
# Configuration for 3D generation
OUTPUT_FILE = "image_to_3d_result.glb"  # Change this to customize output filename
SEED = 0  # Change this for different variations (0 = random)

# Check if image is uploaded
if uploaded_image_data is None:
    print("❌ Please upload an image first using the widget above!")
else:
    print(f"🖼️ Processing image: {uploaded_image_name}")
    print(f"💾 Output file: {OUTPUT_FILE}")
    print(f"🎲 Seed: {SEED}")
    
    try:
        # Convert image to base64 for API request
        image_base64 = base64.b64encode(uploaded_image_data).decode('utf-8')
        
        # Create the JSON payload for image-to-3D
        json_payload = json.dumps({
            "image": image_base64,
            "seed": SEED
        })
        
        print("\n📡 Sending image-to-3D generation request to Trellis NIM...")
        print("⏳ This may take 2-5 minutes...")
        
        # Execute curl command for image-to-3D inference
        curl_cmd = [
            'curl', '-X', 'POST', 'http://localhost:8000/v1/infer',
            '-H', 'Accept: application/json',
            '-H', 'Content-Type: application/json',
            '-d', json_payload,
            '--silent'
        ]
        
        # Run the curl command
        result = subprocess.run(curl_cmd, capture_output=True, text=True, timeout=600)  # 10 minute timeout
        
        if result.returncode != 0:
            print(f"❌ Curl command failed with return code: {result.returncode}")
            print(f"Error: {result.stderr}")
            print(f"Command: {' '.join(curl_cmd)}")
        else:
            response_text = result.stdout.strip()
            print(f"📨 Raw response length: {len(response_text)} characters")
            
            if not response_text:
                print("❌ Empty response from server")
            elif response_text.startswith('<!DOCTYPE') or '<html' in response_text:
                print("❌ Received HTML response instead of JSON - check if container supports image-to-3D")
                print("💡 Some NIM versions may only support text-to-3D")
            else:
                try:
                    # Parse JSON response
                    response_data = json.loads(response_text)
                    print("✅ Valid JSON response received")
                    
                    # Check for artifacts in response
                    if 'artifacts' in response_data and len(response_data['artifacts']) > 0:
                        artifact = response_data['artifacts'][0]
                        if 'base64' in artifact:
                            base64_data = artifact['base64']
                            
                            # Decode and save the GLB file
                            glb_data = base64.b64decode(base64_data)
                            
                            with open(OUTPUT_FILE, 'wb') as f:
                                f.write(glb_data)
                            
                            print(f"✅ SUCCESS! 3D model generated from your image!")
                            print(f"💾 3D model saved as: {OUTPUT_FILE}")
                            print(f"📊 File size: {len(glb_data) / 1024:.2f} KB")
                            print(f"🎯 You can now open {OUTPUT_FILE} in any 3D viewer or Blender!")
                            
                            # Show file info
                            print(f"\n📁 Generated files:")
                            !ls -la *.glb
                            
                        else:
                            print("❌ No base64 data in artifact")
                            print(f"Artifact keys: {list(artifact.keys())}")
                    else:
                        print("❌ No artifacts found in response")
                        print(f"Response keys: {list(response_data.keys())}")
                        if 'error' in response_data:
                            print(f"Error from server: {response_data['error']}")
                        
                        # Show full response for debugging
                        print(f"\n🔍 Full response for debugging:")
                        print(json.dumps(response_data, indent=2)[:1000])
                            
                except json.JSONDecodeError as e:
                    print(f"❌ Failed to parse JSON response: {e}")
                    print(f"Raw response preview: {response_text[:500]}...")

    except subprocess.TimeoutExpired:
        print("❌ Request timed out (>10 minutes)")
        print("💡 Image-to-3D generation may take longer than text-to-3D")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        import traceback
        traceback.print_exc()


## 🎉 Success!

### To generate more 3D models from images:
1. Upload a new image using the file upload widget above
2. Optionally change the `OUTPUT_FILE` and `SEED` in the generation cell
3. Run the generation cell again

### Tips for best results:
- Use images with clear, well-lit subjects
- Objects with distinct shapes work better than complex scenes
- Single objects tend to generate better 3D models than cluttered images
- Try different seeds for variations of the same image

### Image format support:
- **Recommended**: PNG, JPG/JPEG
- **Also supported**: BMP, TIFF, WebP
- **Resolution**: Higher resolution images may produce better results

---


## Utility Commands

Helpful commands for managing your 3D generation workflow:


In [ ]:
# List all generated GLB files
print("📁 Generated 3D model files:")
!ls -la *.glb 2>/dev/null || echo "No GLB files found"


In [ ]:
# Check NIM container status and logs
print("🐳 Container status:")
!docker ps | grep nim-server
print("\n📊 Recent container logs:")
!docker logs nim-server --tail 10


In [ ]:
# Test image upload functionality
print("🧪 Testing upload state:")
if uploaded_image_data is not None:
    print(f"✅ Image loaded: {uploaded_image_name}")
    print(f"📊 Size: {len(uploaded_image_data) / 1024:.2f} KB")
    
    # Show image info
    try:
        image = Image.open(io.BytesIO(uploaded_image_data))
        print(f"📏 Dimensions: {image.width} x {image.height}")
        print(f"🎨 Format: {image.format}")
        print(f"🌈 Mode: {image.mode}")
    except Exception as e:
        print(f"❌ Error reading image: {e}")
else:
    print("❌ No image currently loaded")
    print("💡 Upload an image using the widget above")


In [ ]:
# Create output directory for organized file management
output_dir = "output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"📁 Created output directory: {output_dir}/")
else:
    print(f"📁 Output directory exists: {output_dir}/")

# Move GLB files to output directory
glb_files = !ls *.glb 2>/dev/null || echo ""
if glb_files and glb_files[0]:  # Check if any GLB files exist
    print("\n📦 Moving GLB files to output directory...")
    !mv *.glb output/ 2>/dev/null
    print("✅ GLB files moved to output/")
    !ls -la output/*.glb
else:
    print("\n📭 No GLB files to move")
